#### Adopte1Arbre - notebook - dataset exploration

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

import logging
logging.getLogger("matplotlib.font_manager").setLevel(logging.ERROR)

import dtale
import dtale.global_state as global_state
global_state.set_chart_settings({'scatter_points': 20000, '3d_points': 40000})

import os
import json

In [34]:
os.chdir(r'C:\Users\Jean\Documents\GitHub\Adopte1Arbre')

In [39]:
df=pd.read_csv('les-arbres-plantes.csv', sep=';')

In [9]:
dtale.show(df)

#### review every columns  
- 5734 arbres plantés
- créer des catégories : "tout petit tout mignon", "à taille humaine", "un grand spécial calin" à partir de la circonférence et de la hauteur
- un arbre de 110 m ???
- les rares (<100)

In [10]:
df.shape

(5734, 14)

In [11]:
df["Emplacement - Arrondissement"].value_counts()

PARIS 13E ARRDT      610
PARIS 12E ARRDT      545
PARIS 16E ARRDT      475
PARIS 15E ARRDT      436
PARIS 20E ARRDT      384
SEINE-SAINT-DENIS    374
PARIS 14E ARRDT      368
PARIS 18E ARRDT      354
PARIS 19E ARRDT      260
PARIS 11E ARRDT      244
PARIS 17E ARRDT      242
PARIS 7E ARRDT       234
PARIS 8E ARRDT       229
VAL-DE-MARNE         197
PARIS 10E ARRDT      149
HAUTS-DE-SEINE       139
PARIS 5E ARRDT        98
BOIS DE BOULOGNE      87
BOIS DE VINCENNES     67
PARIS 4E ARRDT        59
PARIS 6E ARRDT        56
PARIS 9E ARRDT        50
PARIS 1ER ARRDT       27
PARIS 3E ARRDT        27
PARIS 2E ARRDT        23
Name: Emplacement - Arrondissement, dtype: int64

In [13]:
df["Arbre - Circonférence en cm"].value_counts()

20.0     4399
15.0      299
10.0      285
25.0      268
14.0       80
16.0       67
30.0       56
18.0       53
5.0        33
35.0       28
0.0        25
23.0       20
22.0       16
12.0       13
21.0        9
6.0         7
26.0        6
24.0        5
40.0        4
27.0        4
45.0        4
75.0        4
80.0        3
28.0        3
19.0        3
13.0        3
31.0        2
50.0        2
70.0        2
17.0        2
59.0        2
100.0       2
65.0        2
4.0         2
3.0         2
1.0         2
55.0        2
185.0       1
95.0        1
90.0        1
46.0        1
60.0        1
110.0       1
135.0       1
128.0       1
97.0        1
160.0       1
2.0         1
67.0        1
140.0       1
130.0       1
113.0       1
Name: Arbre - Circonférence en cm, dtype: int64

In [14]:
df["Arbre - Hauteur en m"].value_counts()

5.0      4450
3.0       539
4.0       317
2.0       161
1.0       118
6.0        87
0.0        25
7.0        12
12.0        7
15.0        6
10.0        4
11.0        2
17.0        2
110.0       1
9.0         1
23.0        1
8.0         1
Name: Arbre - Hauteur en m, dtype: int64

In [25]:
df["Arbre Essence - Nom français"].value_counts().loc["Platane"]

923

#### fonctions de préparation de filtres

In [48]:
def rareOupas(df):
    df_ok=df.copy().reset_index(drop=True)
    df_Essence=df["Arbre Essence - Nom français"].value_counts()
    df_ok["rareOupas"]=["rare" if df_Essence.loc[df_ok["Arbre Essence - Nom français"].iloc[i]]<100 else "frequent" for i in range(df_ok.shape[0])]
    return df_ok

In [36]:
def classTaille(df):
    df_ok=df.copy()
    bins = [0, 3, 5, 10,20]
    labels = ["petit","moyen","grand", "geant"]
    df_ok['classeTaille'] = pd.cut(df_ok["Arbre - Hauteur en m"], bins=bins, labels=labels)
    return df_ok

In [40]:
df_ok=rareOupas(df)

In [41]:
df_ok.shape

(5631, 15)

In [42]:
df_ok

,Emplacement - Identifiant unique,Emplacement - Domanialité gestionnaire,Emplacement - Arrondissement,Emplacement - Complément d'adresse,Emplacement - Site / Adresse,Arbre - Circonférence en cm,Arbre - Hauteur en m,Arbre essence - Espèce var.,Arbre essence - Cultivar,Arbre essence - Genre,Arbre Exploitation - Planté le,Arbre Essence - Nom français,geo_shape,geo_point_2d,rareOupas
0,2039891,Jardin,PARIS 12E ARRDT,NaN,JARDINIERE DE LA RUE JACQUES HILLAIRET,15.0,3.0,antarctica,NaN,Nothofagus,2023-02-01T02:00:00+01:00,Nothofagus,"{""coordinates"": [2.3896969976249585, 48.843784...","48.84378486022892, 2.3896969976249585",rare
1,348177,Alignement,PARIS 17E ARRDT,N°1,BOULEVARD PEREIRE,23.0,4.0,ornus,NaN,Fraxinus,2021-12-03T02:00:00+01:00,Frêne à fleurs,"{""coordinates"": [2.313351848419123, 48.8874823...","48.88748235172613, 2.313351848419123",rare
2,145367,Cimetière,SEINE-SAINT-DENIS,NaN,CIMETIERE DE SAINT-OUEN ANCIEN / AVENUE DE L''...,20.0,4.0,cordata,''Greenspire'',Tilia,2022-01-10T02:00:00+01:00,Tilleul,"{""coordinates"": [2.3509489697880204, 48.906538...","48.90653833095531, 2.3509489697880204",frequent
3,256290,Alignement,PARIS 14E ARRDT,116,AVENUE DU GENERAL LECLERC,25.0,5.0,x hispanica,NaN,Platanus,2022-01-20T02:00:00+01:00,Platane,"{""coordinates"": [2.325937752354315, 48.8253767...","48.825376731279526, 2.325937752354315",frequent
4,2035018,Alignement,PARIS 18E ARRDT,NaN,RUE FERDINAND FLOCON,20.0,5.0,carpinifolia,NaN,Ostrya,2021-12-23T02:00:00+01:00,Charme-Houblon,"{""coordinates"": [2.345930879134525, 48.8916226...","48.89162264234357, 2.345930879134525",frequent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5626,2040728,DASCO,PARIS 18E ARRDT,NaN,ECOLE ELEMENTAIRE / 33 RUE DE L EVANGILE,20.0,5.0,australis,NaN,Celtis,2023-03-01T02:00:00+01:00,Micocoulier,"{""coordinates"": [2.3650679085758384, 48.894450...","48.894450977140366, 2.3650679085758384",frequent
5627,199643,DJS,PARIS 20E ARRDT,BOULODROME,CENTRE SPORTIF DOCTEURS DEJERINE / 36 RUE DES ...,20.0,5.0,campestre,''Elsrijk'',Acer,2022-02-21T02:00:00+01:00,Erable,"{""coordinates"": [2.412440607282452, 48.8552224...","48.8552224410884, 2.412440607282452",frequent
5628,268337,Alignement,PARIS 11E ARRDT,8,RUE DU GENERAL RENAULT,20.0,5.0,hippocastanum,''Baumannii'',Aesculus,2022-01-13T02:00:00+01:00,Marronnier,"{""coordinates"": [2.378364905304499, 48.8612889...","48.861288928546706, 2.378364905304499",frequent
5629,2031599,Alignement,PARIS 5E ARRDT,NaN,PORT DE LA TOURNELLE,20.0,5.0,alba,''Raket'',Populus,2022-02-14T02:00:00+01:00,Peuplier,"{""coordinates"": [2.3521724753846245, 48.851115...","48.851115269136265, 2.3521724753846245",rare


In [43]:
df_ok=classTaille(df_ok)

In [44]:
df_ok.head()

,Emplacement - Identifiant unique,Emplacement - Domanialité gestionnaire,Emplacement - Arrondissement,Emplacement - Complément d'adresse,Emplacement - Site / Adresse,Arbre - Circonférence en cm,Arbre - Hauteur en m,Arbre essence - Espèce var.,Arbre essence - Cultivar,Arbre essence - Genre,Arbre Exploitation - Planté le,Arbre Essence - Nom français,geo_shape,geo_point_2d,rareOupas,classeTaille
0,2039891,Jardin,PARIS 12E ARRDT,NaN,JARDINIERE DE LA RUE JACQUES HILLAIRET,15.0,3.0,antarctica,NaN,Nothofagus,2023-02-01T02:00:00+01:00,Nothofagus,"{""coordinates"": [2.3896969976249585, 48.843784...","48.84378486022892, 2.3896969976249585",rare,petit
1,348177,Alignement,PARIS 17E ARRDT,N°1,BOULEVARD PEREIRE,23.0,4.0,ornus,NaN,Fraxinus,2021-12-03T02:00:00+01:00,Frêne à fleurs,"{""coordinates"": [2.313351848419123, 48.8874823...","48.88748235172613, 2.313351848419123",rare,moyen
2,145367,Cimetière,SEINE-SAINT-DENIS,NaN,CIMETIERE DE SAINT-OUEN ANCIEN / AVENUE DE L''...,20.0,4.0,cordata,''Greenspire'',Tilia,2022-01-10T02:00:00+01:00,Tilleul,"{""coordinates"": [2.3509489697880204, 48.906538...","48.90653833095531, 2.3509489697880204",frequent,moyen
3,256290,Alignement,PARIS 14E ARRDT,116,AVENUE DU GENERAL LECLERC,25.0,5.0,x hispanica,NaN,Platanus,2022-01-20T02:00:00+01:00,Platane,"{""coordinates"": [2.325937752354315, 48.8253767...","48.825376731279526, 2.325937752354315",frequent,moyen
4,2035018,Alignement,PARIS 18E ARRDT,NaN,RUE FERDINAND FLOCON,20.0,5.0,carpinifolia,NaN,Ostrya,2021-12-23T02:00:00+01:00,Charme-Houblon,"{""coordinates"": [2.345930879134525, 48.8916226...","48.89162264234357, 2.345930879134525",frequent,moyen


In [45]:
df_ok=df_ok.sort_index()
df_ok["coordGeo"]=[[float(my_val) for my_val in df_ok["geo_point_2d"].iloc[i].split(",")] for i in range(df_ok.shape[0])]
df_ok["lat"]=[float(df_ok["geo_point_2d"].iloc[i].split(",")[0]) for i in range(df_ok.shape[0])]
df_ok["lon"]=[float(df_ok["geo_point_2d"].iloc[i].split(",")[1]) for i in range(df_ok.shape[0])]
df_ok["date_plantation"]=pd.to_datetime(df_ok["Arbre Exploitation - Planté le"])
df_ok["mois_plantation"]=[my_date.month for my_date in df_ok["date_plantation"]]
df_ok["jour_plantation"]=[my_date.day for my_date in df_ok["date_plantation"]]
df_ok.to_csv('les-arbres-plantes-clean.csv', sep=';')

In [86]:
df_ok

,Emplacement - Identifiant unique,Emplacement - Domanialité gestionnaire,Emplacement - Arrondissement,Emplacement - Complément d'adresse,Emplacement - Site / Adresse,Arbre - Circonférence en cm,Arbre - Hauteur en m,Arbre essence - Espèce var.,Arbre essence - Cultivar,Arbre essence - Genre,Arbre Exploitation - Planté le,Arbre Essence - Nom français,geo_shape,geo_point_2d,rareOupas,classeTaille,coordGeo,lat,lon
0,2039891,Jardin,PARIS 12E ARRDT,NaN,JARDINIERE DE LA RUE JACQUES HILLAIRET,15.0,3.0,antarctica,NaN,Nothofagus,2023-02-01T02:00:00+01:00,Nothofagus,"{""coordinates"": [2.3896969976249585, 48.843784...","48.84378486022892, 2.3896969976249585",rare,petit,"[48.84378486022892, 2.3896969976249585]",48.843785,2.389697
1,348177,Alignement,PARIS 17E ARRDT,N°1,BOULEVARD PEREIRE,23.0,4.0,ornus,NaN,Fraxinus,2021-12-03T02:00:00+01:00,Frêne à fleurs,"{""coordinates"": [2.313351848419123, 48.8874823...","48.88748235172613, 2.313351848419123",rare,moyen,"[48.88748235172613, 2.313351848419123]",48.887482,2.313352
2,145367,Cimetière,SEINE-SAINT-DENIS,NaN,CIMETIERE DE SAINT-OUEN ANCIEN / AVENUE DE L''...,20.0,4.0,cordata,''Greenspire'',Tilia,2022-01-10T02:00:00+01:00,Tilleul,"{""coordinates"": [2.3509489697880204, 48.906538...","48.90653833095531, 2.3509489697880204",frequent,moyen,"[48.90653833095531, 2.3509489697880204]",48.906538,2.350949
3,256290,Alignement,PARIS 14E ARRDT,116,AVENUE DU GENERAL LECLERC,25.0,5.0,x hispanica,NaN,Platanus,2022-01-20T02:00:00+01:00,Platane,"{""coordinates"": [2.325937752354315, 48.8253767...","48.825376731279526, 2.325937752354315",frequent,moyen,"[48.825376731279526, 2.325937752354315]",48.825377,2.325938
4,2035018,Alignement,PARIS 18E ARRDT,NaN,RUE FERDINAND FLOCON,20.0,5.0,carpinifolia,NaN,Ostrya,2021-12-23T02:00:00+01:00,Charme-Houblon,"{""coordinates"": [2.345930879134525, 48.8916226...","48.89162264234357, 2.345930879134525",frequent,moyen,"[48.89162264234357, 2.345930879134525]",48.891623,2.345931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5626,2040728,DASCO,PARIS 18E ARRDT,NaN,ECOLE ELEMENTAIRE / 33 RUE DE L EVANGILE,20.0,5.0,australis,NaN,Celtis,2023-03-01T02:00:00+01:00,Micocoulier,"{""coordinates"": [2.3650679085758384, 48.894450...","48.894450977140366, 2.3650679085758384",frequent,moyen,"[48.894450977140366, 2.3650679085758384]",48.894451,2.365068
5627,199643,DJS,PARIS 20E ARRDT,BOULODROME,CENTRE SPORTIF DOCTEURS DEJERINE / 36 RUE DES ...,20.0,5.0,campestre,''Elsrijk'',Acer,2022-02-21T02:00:00+01:00,Erable,"{""coordinates"": [2.412440607282452, 48.8552224...","48.8552224410884, 2.412440607282452",frequent,moyen,"[48.8552224410884, 2.412440607282452]",48.855222,2.412441
5628,268337,Alignement,PARIS 11E ARRDT,8,RUE DU GENERAL RENAULT,20.0,5.0,hippocastanum,''Baumannii'',Aesculus,2022-01-13T02:00:00+01:00,Marronnier,"{""coordinates"": [2.378364905304499, 48.8612889...","48.861288928546706, 2.378364905304499",frequent,moyen,"[48.861288928546706, 2.378364905304499]",48.861289,2.378365
5629,2031599,Alignement,PARIS 5E ARRDT,NaN,PORT DE LA TOURNELLE,20.0,5.0,alba,''Raket'',Populus,2022-02-14T02:00:00+01:00,Peuplier,"{""coordinates"": [2.3521724753846245, 48.851115...","48.851115269136265, 2.3521724753846245",rare,moyen,"[48.851115269136265, 2.3521724753846245]",48.851115,2.352172


In [64]:
type(df_ok["coordGeo"].iloc[0][0])

float

In [69]:
type(df_ok["Emplacement - Identifiant unique"].iloc[0])

numpy.int64

In [87]:
df_ok[df_ok["Emplacement - Identifiant unique"]==120421]

,Emplacement - Identifiant unique,Emplacement - Domanialité gestionnaire,Emplacement - Arrondissement,Emplacement - Complément d'adresse,Emplacement - Site / Adresse,Arbre - Circonférence en cm,Arbre - Hauteur en m,Arbre essence - Espèce var.,Arbre essence - Cultivar,Arbre essence - Genre,Arbre Exploitation - Planté le,Arbre Essence - Nom français,geo_shape,geo_point_2d,rareOupas,classeTaille,coordGeo,lat,lon
3994,120421,Jardin,PARIS 18E ARRDT,NaN,SQUARE DES DEUX NETHES / 24 AVENUE DE CLICHY,20.0,5.0,sylvestris,NaN,Malus,2022-02-03T02:00:00+01:00,Pommier à fleurs,"{""coordinates"": [2.326915884212205, 48.8854478...","48.88544784192849, 2.326915884212205",rare,moyen,"[48.88544784192849, 2.326915884212205]",48.885448,2.326916


In [88]:
df_ok[df_ok["Emplacement - Identifiant unique"]==120421].loc[:,"Arbre Exploitation - Planté le"]

3994    2022-02-03T02:00:00+01:00
Name: Arbre Exploitation - Planté le, dtype: object

In [89]:
df_ok["date_plantation"]=pd.to_datetime(df_ok["Arbre Exploitation - Planté le"])

In [95]:
df_ok.head()

,Emplacement - Identifiant unique,Emplacement - Domanialité gestionnaire,Emplacement - Arrondissement,Emplacement - Complément d'adresse,Emplacement - Site / Adresse,Arbre - Circonférence en cm,Arbre - Hauteur en m,Arbre essence - Espèce var.,Arbre essence - Cultivar,Arbre essence - Genre,...,geo_shape,geo_point_2d,rareOupas,classeTaille,coordGeo,lat,lon,date_plantation,mois_plantation,jour_plantation
0,2039891,Jardin,PARIS 12E ARRDT,NaN,JARDINIERE DE LA RUE JACQUES HILLAIRET,15.0,3.0,antarctica,NaN,Nothofagus,...,"{""coordinates"": [2.3896969976249585, 48.843784...","48.84378486022892, 2.3896969976249585",rare,petit,"[48.84378486022892, 2.3896969976249585]",48.843785,2.389697,2023-02-01 02:00:00+01:00,2,1
1,348177,Alignement,PARIS 17E ARRDT,N°1,BOULEVARD PEREIRE,23.0,4.0,ornus,NaN,Fraxinus,...,"{""coordinates"": [2.313351848419123, 48.8874823...","48.88748235172613, 2.313351848419123",rare,moyen,"[48.88748235172613, 2.313351848419123]",48.887482,2.313352,2021-12-03 02:00:00+01:00,12,3
2,145367,Cimetière,SEINE-SAINT-DENIS,NaN,CIMETIERE DE SAINT-OUEN ANCIEN / AVENUE DE L''...,20.0,4.0,cordata,''Greenspire'',Tilia,...,"{""coordinates"": [2.3509489697880204, 48.906538...","48.90653833095531, 2.3509489697880204",frequent,moyen,"[48.90653833095531, 2.3509489697880204]",48.906538,2.350949,2022-01-10 02:00:00+01:00,1,10
3,256290,Alignement,PARIS 14E ARRDT,116,AVENUE DU GENERAL LECLERC,25.0,5.0,x hispanica,NaN,Platanus,...,"{""coordinates"": [2.325937752354315, 48.8253767...","48.825376731279526, 2.325937752354315",frequent,moyen,"[48.825376731279526, 2.325937752354315]",48.825377,2.325938,2022-01-20 02:00:00+01:00,1,20
4,2035018,Alignement,PARIS 18E ARRDT,NaN,RUE FERDINAND FLOCON,20.0,5.0,carpinifolia,NaN,Ostrya,...,"{""coordinates"": [2.345930879134525, 48.8916226...","48.89162264234357, 2.345930879134525",frequent,moyen,"[48.89162264234357, 2.345930879134525]",48.891623,2.345931,2021-12-23 02:00:00+01:00,12,23


#### fonction pour se connecter à l'API et charger le dataset actualisé - si nouveau

In [22]:
import requests
import json
import pandas as pd
from pandas import json_normalize

In [23]:
def get_dataset():
    '''
    get dataset last update
    '''
    #use of API  
    dataset_id="les-arbres-plantes"
    base_request_string=f"https://opendata.paris.fr/api/records/1.0/search/?dataset={dataset_id}&q=&rows=10000"
    r = requests.get(base_request_string)
    dataset_loaded = json.loads(r.content)

    #format dataset
    dataset_pandas=json_normalize(dataset_loaded['records'])
    return dataset_pandas

In [25]:
df_new=get_dataset()

In [26]:
df_new.head(5)

,datasetid,recordid,record_timestamp,fields.arrondissement,fields.espece,fields.circonferenceencm,fields.dateplantation,fields.domanialite,fields.geo_point_2d,fields.hauteurenm,fields.genre,fields.libellefrancais,fields.geo_shape.coordinates,fields.geo_shape.type,fields.idbase,fields.adresse,geometry.type,geometry.coordinates,fields.varieteoucultivar,fields.complementadresse
0,les-arbres-plantes,e7d2393b7f60b7cc4162388f8df6c37515c9e269,2023-08-21T04:50:08.906Z,PARIS 20E ARRDT,trichotomum var. fargesii,20.0,2022-03-15T01:00:00+00:00,Alignement,"[48.86904358791986, 2.4047522297453847]",4.0,Clerodendrum,Clerodendron,"[2.4047522297453847, 48.86904358791986]",Point,2035840,RUE DU GROUPE MANOUCHIAN,Point,"[2.4047522297453847, 48.86904358791986]",NaN,NaN
1,les-arbres-plantes,c0b3d78afeca6af7304be4caf92d96d6cc022bcf,2023-08-21T04:50:08.906Z,PARIS 17E ARRDT,indica,14.0,2022-12-22T01:00:00+00:00,DJS,"[48.896822162746815, 2.3126304123346286]",2.0,Lagerstroemia,Lilas des indes,"[2.3126304123346286, 48.896822162746815]",Point,2040378,CENTRE SPORTIF LEON BIANCOTTO / 6 AVENUE DE LA...,Point,"[2.3126304123346286, 48.896822162746815]",''Desmou'' LA MOUSSON,NaN
2,les-arbres-plantes,b4793852cb8b1ddb82dbd4be97845d41cd87ee1b,2023-08-21T04:50:08.906Z,PARIS 17E ARRDT,x hispanica,20.0,2022-02-22T01:00:00+00:00,Alignement,"[48.881796560018124, 2.2808669338577348]",5.0,Platanus,Platane,"[2.2808669338577348, 48.881796560018124]",Point,296821,AVENUE DE LA PORTE DES TERNES,Point,"[2.2808669338577348, 48.881796560018124]",NaN,N°30
3,les-arbres-plantes,a61227fc0d13340b373cae9bc9d3705eb103c5fc,2023-08-21T04:50:08.906Z,PARIS 7E ARRDT,x hispanica,20.0,2022-03-14T01:00:00+00:00,Alignement,"[48.86161166786992, 2.297090208255415]",5.0,Platanus,Platane,"[2.297090208255415, 48.86161166786992]",Point,258455,QUAI BRANLY,Point,"[2.297090208255415, 48.86161166786992]",NaN,NaN
4,les-arbres-plantes,c63266c729163829a4c24de80f15732e84045e94,2023-08-21T04:50:08.906Z,PARIS 7E ARRDT,x hispanica,20.0,2022-02-23T01:00:00+00:00,Alignement,"[48.8479263070361, 2.3107823473955564]",5.0,Platanus,Platane,"[2.3107823473955564, 48.8479263070361]",Point,218525,AVENUE DE SAXE,Point,"[2.3107823473955564, 48.8479263070361]",NaN,41


In [28]:
df_new2=pd.read_csv("https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/les-arbres-plantes/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B", sep=';')

In [29]:
df_new2.shape

(5809, 14)

In [30]:
df_new2.head()

,Emplacement - Identifiant unique,Emplacement - Domanialité gestionnaire,Emplacement - Arrondissement,Emplacement - Complément d'adresse,Emplacement - Site / Adresse,Arbre - Circonférence en cm,Arbre - Hauteur en m,Arbre essence - Espèce var.,Arbre essence - Cultivar,Arbre essence - Genre,Arbre Exploitation - Planté le,Arbre Essence - Nom français,geo_shape,geo_point_2d
0,226425,Alignement,PARIS 18E ARRDT,40,BOULEVARD BARBES,20.0,5.0,x hispanica,NaN,Platanus,2022-03-09T02:00:00+01:00,Platane,"{""coordinates"": [2.34970430984417, 48.88679640...","48.886796405910154, 2.34970430984417"
1,204148,Alignement,PARIS 12E ARRDT,28,AVENUE DU BEL AIR,20.0,5.0,platanoides,''Columnare'',Acer,2023-03-29T04:00:00+02:00,Erable,"{""coordinates"": [2.396990547166306, 48.8472162...","48.84721626693131, 2.396990547166306"
2,302382,Alignement,PARIS 9E ARRDT,16,BOULEVARD DES ITALIENS,20.0,5.0,australis,NaN,Celtis,2022-12-01T02:00:00+01:00,Micocoulier,"{""coordinates"": [2.3378854490129926, 48.871705...","48.87170559215565, 2.3378854490129926"
3,227773,Alignement,PARIS 20E ARRDT,NaN,RUE DE BUZENVAL,20.0,5.0,toringo,NaN,Malus,2021-12-06T02:00:00+01:00,Pommier à fleurs,"{""coordinates"": [2.3989153824452383, 48.854996...","48.854996779916796, 2.3989153824452383"
4,2017070,Alignement,PARIS 18E ARRDT,23,RUE MULLER,20.0,5.0,pseudoacacia,NaN,Robinia,2023-02-28T02:00:00+01:00,Robinier,"{""coordinates"": [2.3450383545878863, 48.886347...","48.88634732910571, 2.3450383545878863"


In [49]:
def build_df_clean(df):
    df_ok=rareOupas(df)
    df_ok=classTaille(df_ok)
    df_ok=df_ok.sort_index()
    df_ok["coordGeo"]=[[float(my_val) for my_val in df_ok["geo_point_2d"].iloc[i].split(",")] for i in range(df_ok.shape[0])]
    df_ok["lat"]=[float(df_ok["geo_point_2d"].iloc[i].split(",")[0]) for i in range(df_ok.shape[0])]
    df_ok["lon"]=[float(df_ok["geo_point_2d"].iloc[i].split(",")[1]) for i in range(df_ok.shape[0])]
    df_ok["date_plantation"]=pd.to_datetime(df_ok["Arbre Exploitation - Planté le"])
    df_ok["mois_plantation"]=[my_date.month for my_date in df_ok["date_plantation"]]
    df_ok["jour_plantation"]=[my_date.day for my_date in df_ok["date_plantation"]]
    print(df_ok.shape)
    df_ok.to_csv('les-arbres-plantes-clean.csv', sep=';')
    return "all good"

In [50]:
build_df_clean(df_new2)

KeyError: nan